In [1]:
from music21 import converter, instrument, note, chord, stream
import numpy
from matplotlib import pyplot as plt
import keras
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from keras import Sequential, Model
from keras.layers import LSTM, Dropout, Dense, Activation, Input, concatenate
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Embedding
import tensorflow as tf
from tqdm import tqdm
import numpy as np

In [2]:
!unzip dataset.zip

Archive:  dataset.zip
replace dataset/chopin/chpn-p12.mid? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace dataset/chopin/chpn-p12.mid? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/chopin/chpn-p12.mid  
  inflating: dataset/chopin/chpn-p15.mid  
  inflating: dataset/chopin/chpn-p16.mid  
  inflating: dataset/chopin/chpn-p17.mid  
  inflating: dataset/chopin/chpn-p18.mid  
  inflating: dataset/chopin/chpn-p19.mid  
  inflating: dataset/chopin/chpn-p20.mid  
  inflating: dataset/chopin/chpn-p21.mid  
  inflating: dataset/chopin/chpn-p22.mid  
  inflating: dataset/chopin/chpn-p23.mid  
  inflating: dataset/chopin/chpn-p24.mid  
  inflating: dataset/chopin/chpn-p7.mid  
  inflating: dataset/chopin/chpn_op10_e01.mid  
  inflating: dataset/chopin/chpn_op10_e05.mid  
  inflating: dataset/chopin/chpn_op10_e12.mid  
  inflating: dataset/chopin/chpn_op23.mid  
  inflating: dataset/chopin/chpn_op25_e1.mid  
  inflating: dataset/chopin/chpn_op25_e2.mid 

In [3]:
def get_notes(midi):
  notes = []

  notes_from_midi = midi.flatten().notesAndRests
  notes_sorted = sorted(notes_from_midi, key=lambda note: note.offset)

  prev_start = notes_sorted[0].offset

  for element in notes_sorted:
    if isinstance(element , note.Note):
      notes.append((float(element.offset), float(element.volume.velocity),float(element.seconds), str(element.pitch), float(element.offset-prev_start)))
    elif isinstance(element, chord.Chord):
      #notes.append((float(element.offset), float(element.volume.velocity), float(element.seconds), '+'.join(str(n) for n in element.normalOrder)))
      for noteChord in element.pitches:
        notes.append((float(element.offset),float(element.volume.velocity), float(element.seconds), str(noteChord), float(element.offset-prev_start)))

    elif isinstance(element, note.Rest):
        # Bizarre car main gauche et droite flatten (réunie)
        # Voir autre implémentation
        notes.append((float(element.offset), 0,float(element.seconds), "REST", float(element.offset-prev_start)))

    prev_start = element.offset
  return notes



In [17]:
input_note, output_note = [], []
input_offset, output_offset = [], []
input_volume, output_volume = [], []
input_duration, output_duration = [], []

note_all_songs = []
for folder in tqdm(os.walk("dataset")):
    if len(folder) != 3:
      continue
    else:
      for file in folder[2][:1]:
        file_path = str(folder[0])+"/"+file

        # convertit la piste midi
        midi = converter.parse(file_path)

        # récupère les notes, volume, dureées...
        notes = get_notes(midi)

        # ajoute à une liste globale
        note_all_songs.append(notes)

3it [00:06,  2.10s/it]


In [18]:
# fais un dataframe à partir de toutes les notes (sans distinctions)
df = pd.DataFrame([], columns=['debut_note','volume','durée(s)', 'pitch/chord', 'offset'])
df = df.reset_index(drop=True)
for elem in note_all_songs:
    df2 = pd.DataFrame(elem, columns=['debut_note','volume','durée(s)', 'pitch/chord', 'offset'])
    df2 = df2.reset_index(drop=True)
    df =pd.concat([df,df2])
    df = df.reset_index(drop=True)

In [19]:
df

,debut_note,volume,durée(s),pitch/chord,offset
0,0.00,90.0,2.278481,G4,0.00
1,0.00,90.0,2.278481,G5,0.00
2,0.00,65.0,2.278481,G2,0.00
3,0.00,65.0,2.278481,G1,0.00
4,3.00,44.0,0.413755,B4,3.00
...,...,...,...,...,...
6163,81.00,68.0,2.326258,F1,0.00
6164,85.00,0.0,1.543354,REST,4.00
6165,85.00,0.0,1.543354,REST,0.00
6166,87.75,0.0,0.681818,REST,2.75


In [20]:
#train les encoders pour les notes, volumes et durées en catégorielles (one hot)
# sur intégralité des données (obligé)
# Voir si on peut pas enregistré le transformateur
oh_notes =  OneHotEncoder().fit(df[['pitch/chord']])


# seuil/catégories modifiable
dict_volume_class = {
    "very low":10,
    "low":30,
    "low medium":50,
    "medium":70,
    "high": 90,
    "very high":128
}
bins_volume = list(dict_volume_class.values())
bins_volume.insert(0,-0.1)
values_volume = list(dict_volume_class.keys())
df["volume_class"] = pd.cut(df["volume"], bins=bins_volume, labels=values_volume)
"""df["volume_class_interval"] = pd.cut(df["volume"], bins=20)
intervals = df["volume_class_interval"]
median_values = [(interval.left + interval.right) / 2 for interval in intervals]
df["volume_class"] = median_values"""
oh_volume =  OneHotEncoder().fit(df[['volume_class']])


MinMaxScaler


dict_duration_class = {
"very short" : 0.1,
"short" :0.25,
"medium":0.5,
"medium-long":0.75,
"long":1.0,
"very long": 1.5,
"kilometer long":2.0,
"yearlight distance":3.0,
"super mega long":df['durée(s)'].max()
}
bins_duration = list(dict_duration_class.values())
bins_duration.insert(0,-0.1)
values_duration = list(dict_duration_class.keys())
df["duration_class"] = pd.cut(df["durée(s)"], bins=bins_duration, labels=values_duration)
"""df["duration_class_interval"] = pd.cut(df["durée(s)"], bins=20)
intervals = df["duration_class_interval"]
median_values = [(interval.left + interval.right) / 2 for interval in intervals]
df["duration_class"] = median_values"""
oh_duration =  OneHotEncoder().fit(df[['duration_class']])


In [21]:
def prepare_sequence(data, sequence_length, network_input, network_output):
  # create input sequences and the corresponding outputs
  for i in range(0, len(data) - sequence_length, 1):
      sequence_in = data[i:i + sequence_length]
      sequence_out = data[i + sequence_length]
      network_input.append(sequence_in)
      network_output.append(sequence_out)
  return (network_input, network_output)

def reshape_array_input(array_tensor, sequence_length):
  n_patterns = len(array_tensor)
  array_reshaped = numpy.reshape(array_tensor, (n_patterns, sequence_length, -1))
  return array_reshaped

def reshape_array_output(array_tensor, sequence_length):
  n_patterns = len(array_tensor)
  array_reshaped = numpy.reshape(array_tensor, (len(array_tensor), -1))
  return array_reshaped


In [22]:
sequence_length = 100

In [23]:
def check_pitch(x):
  if ('+' in x) or x.isdigit():
        notes_in_chord = x.split('+')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord =chord.Chord(notes).pitches[0].midi
        return new_chord
    # pattern is a note
  elif ("REST" in x):
    return 0
  else:
        new_note = note.Note(x).pitch.midi
        return new_note

In [24]:
input_note, output_note = [], []
input_offset, output_offset = [], []
input_volume, output_volume = [], []
input_duration, output_duration = [], []
input_step, output_step = [], []
input_pitch_n, output_pitch_n = [], []

# pour chaque music, on la transforme en dataframe, transforme les notes, durées et volumes
for music in note_all_songs:
  df_music = pd.DataFrame(music, columns=['debut_note','volume','durée(s)', 'pitch/chord', 'step'])

  # encode le volume et la durée
  df_music["volume_class"] = pd.cut(df_music["volume"], bins=bins_volume, labels=values_volume)
  df_music["duration_class"] = pd.cut(df_music["durée(s)"], bins=bins_duration, labels=values_duration)

  df_music['pitch_n'] = df_music['pitch/chord'].apply(check_pitch)


  # transform en one hot à partir des modèles entrainés
  notes_encoded = oh_notes.transform(df_music[['pitch/chord']]).toarray()
  volume_encoded = oh_volume.transform(df_music[['volume_class']]).toarray()
  duration_encoded = oh_duration.transform(df_music[['duration_class']]).toarray()

  # prépare les input (longeur = sequence_length) pour chaque morceau
  # garde certaine cohérence au morceau même
  input_offset, output_offset = prepare_sequence(df_music['debut_note'].values, sequence_length, input_offset, output_offset)
  input_note, output_note = prepare_sequence(notes_encoded, sequence_length, input_note, output_note)
  #input_volume, output_volume = prepare_sequence(volume_encoded, sequence_length, input_volume, output_volume)
  input_volume, output_volume = prepare_sequence(df_music["volume"].values, sequence_length, input_volume, output_volume)


  #input_duration, output_duration = prepare_sequence(duration_encoded, sequence_length, input_duration, output_duration)
  input_duration, output_duration = prepare_sequence(df_music["durée(s)"].values, sequence_length, input_duration, output_duration)

  input_step, output_step = prepare_sequence(df_music['step'].values, sequence_length, input_step, output_step)
  input_pitch_n, output_pitch_n = prepare_sequence(df_music['pitch_n'].values, sequence_length, input_pitch_n, output_pitch_n)

In [25]:
# on reshape tout
input_note_reshaped= reshape_array_input(input_note, sequence_length)
output_note_reshaped = reshape_array_output(output_note, sequence_length)

input_offset_reshaped = reshape_array_input(input_offset, sequence_length)
output_offset_reshaped = reshape_array_output(output_offset, sequence_length)

input_volume_reshaped= reshape_array_input(input_volume, sequence_length)
output_volume_reshaped = reshape_array_output(output_volume, sequence_length)

input_volume_reshaped = input_volume_reshaped/127.0
output_volume_reshaped = output_volume_reshaped/127.0

input_duration_reshaped= reshape_array_input(input_duration, sequence_length)
output_duration_reshaped = reshape_array_output(output_duration, sequence_length)


input_step_reshaped= reshape_array_input(input_step, sequence_length)
output_step_reshaped = reshape_array_output(output_step, sequence_length)

input_pitch_n_reshaped = np.reshape(input_pitch_n, (len(input_pitch_n),sequence_length, -1))
output_pitch_n_reshaped = np.reshape(output_pitch_n, (len(output_pitch_n),1))


In [28]:
#création du modèle (à alléger si overfitting)
inputNotes_layer = Input(shape=(input_note_reshaped.shape[1], input_note_reshaped.shape[2]))
inputNotes = LSTM(
        32,
        input_shape=(input_note_reshaped.shape[1], input_note_reshaped.shape[2]),
        return_sequences=True
    )(inputNotes_layer)

inputVolume_layer = Input(shape=(input_volume_reshaped.shape[1], input_volume_reshaped.shape[2]))
inputVolume = LSTM(
        32,
        input_shape=(input_volume_reshaped.shape[1], input_volume_reshaped.shape[2]),
        return_sequences=True
    )(inputVolume_layer)

inputDuration_layer = Input(shape=(input_duration_reshaped.shape[1], input_duration_reshaped.shape[2]))
inputDuration = LSTM(
        32,
        input_shape=(input_duration_reshaped.shape[1], input_duration_reshaped.shape[2]),
        return_sequences=True
    )(inputDuration_layer)

inputOffset_layer = Input(shape=(input_offset_reshaped.shape[1], input_offset_reshaped.shape[2]))
inputOffset = LSTM(
        32,
        input_shape=(input_offset_reshaped.shape[1], input_offset_reshaped.shape[2]),
        return_sequences=True
    )(inputOffset_layer)


inputs = concatenate([inputNotes, inputVolume, inputDuration, inputOffset])
x = LSTM(128, return_sequences=True)(inputs)
#x = Dropout(0.3)(x)
x = LSTM(128)(inputs)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)

outputNotes = Dense(32, activation='relu')(x)
outputNotes = Dropout(0.3)(outputNotes)
outputNotes = Dense(output_note_reshaped.shape[1], activation='softmax', name="Note")(outputNotes)

"""outputNotes = Dense(32, activation='relu')(x)
outputNotes = Dropout(0.3)(outputNotes)
outputNotes = Dense(101, activation='softmax', name="Note")(outputNotes)"""

outputVolume = Dense(32, activation='relu')(x)
outputVolume = Dropout(0.3)(outputVolume)
outputVolume = BatchNormalization()(outputVolume)
outputVolume = Dense(output_volume_reshaped.shape[1], activation='softmax', name="Volume")(outputVolume)

outputDuration = Dense(32, activation='relu')(x)
outputDuration = Dropout(0.3)(outputDuration)
outputDuration = BatchNormalization()(outputDuration)
outputDuration = Dense(output_duration_reshaped.shape[1], activation='softmax', name="Duration")(outputDuration)

outputOffset =  Dense(32, activation='relu')(x)
outputOffset = Dropout(0.3)(outputOffset)
outputOffset = BatchNormalization()(outputOffset)
outputOffset = Dense(output_offset_reshaped.shape[1], name="Offset")(outputOffset)



model = Model(inputs=[inputNotes_layer,inputOffset_layer,  inputVolume_layer, inputDuration_layer], outputs=[outputNotes, outputOffset, outputVolume, outputDuration])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

def mse_with_positive_pressure(y_true: tf.Tensor, y_pred: tf.Tensor):
  mse = (y_true - y_pred) ** 2
  positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
  return tf.reduce_mean(mse + positive_pressure)

#Adam seems to be faster than RMSProp and learns better too
model.compile(loss=["categorical_crossentropy", mse_with_positive_pressure, mse_with_positive_pressure, mse_with_positive_pressure], optimizer=optimizer)

In [ ]:
model.summary()

In [15]:
import os

cp_callback = keras.callbacks.ModelCheckpoint(
    'model_weights_epoch.h5',  # Nom du fichier de sauvegarde avec un espace réservé pour le numéro de l'époque
    save_best_only=True,  # Sauvegarder à chaque époque, pas seulement les meilleurs modèles
    save_weights_only=True,  # Sauvegarder uniquement les poids, pas l'ensemble du modèle
    verbose=1  # Afficher un message lors de la sauvegarde
    )

In [29]:
model.fit([input_note_reshaped, input_step_reshaped, input_volume_reshaped, input_duration_reshaped], [output_note_reshaped, output_step_reshaped, output_volume_reshaped, output_duration_reshaped], epochs=400, callbacks=[cp_callback], batch_size=128)

Epoch 1/400
46/47 [============================>.] - ETA: 0s - loss: 5.1624 - Note_loss: 3.7759 - Offset_loss: 0.3213 - Volume_loss: 0.4726 - Duration_loss: 0.5925

47/47 [==============================] - 14s 33ms/step - loss: 5.1568 - Note_loss: 3.7737 - Offset_loss: 0.3177 - Volume_loss: 0.4728 - Duration_loss: 0.5926
Epoch 2/400
47/47 [==============================] - ETA: 0s - loss: 4.6227 - Note_loss: 3.4896 - Offset_loss: 0.0678 - Volume_loss: 0.4728 - Duration_loss: 0.5926

47/47 [==============================] - 2s 33ms/step - loss: 4.6227 - Note_loss: 3.4896 - Offset_loss: 0.0678 - Volume_loss: 0.4728 - Duration_loss: 0.5926
Epoch 3/400
47/47 [==============================] - ETA: 0s - loss: 4.4965 - Note_loss: 3.3629 - Offset_loss: 0.0682 - Volume_loss: 0.4728 - Duration_loss: 0.5926

47/47 [==============================] - 1s 31ms/step - loss: 4.4965 - Note_loss: 3.3629 - Offset_loss: 0.0682 - Volume_loss: 0.4728 - Duration_loss: 0.5926
Epoch 4/400
47/47 [==============================] - ETA: 0s - loss: 4.4208 - Note_loss: 3.2877 - Offset_loss: 0.0676 - Volume_loss: 0.4728 - Duration_loss: 0.5926

47/47 [==============================] - 1s 30ms/step - loss: 4.4208 - Note_loss: 3.2877 - Offset_loss: 0.0676 - Volume_loss: 0.4728 - Duration_loss: 0.5926
Epoch 5/400
47/47 [==============================] - ETA: 0s - loss: 4.3882 - Note_loss: 3.2554 - Offset_loss: 0.0674 - Volume_loss: 0.4728 - Duration_loss: 0.5926

...

47/47 [==============================] - 2s 34ms/step - loss: 1.5242 - Note_loss: 0.4268 - Offset_loss: 0.0320 - Volume_loss: 0.4728 - Duration_loss: 0.5926
Epoch 147/400
47/47 [==============================] - ETA: 0s - loss: 1.5180 - Note_loss: 0.4188 - Offset_loss: 0.0338 - Volume_loss: 0.4728 - Duration_loss: 0.5926

47/47 [==============================] - 2s 43ms/step - loss: 1.5180 - Note_loss: 0.4188 - Offset_loss: 0.0338 - Volume_loss: 0.4728 - Duration_loss: 0.5926
Epoch 148/400
47/47 [==============================] - ETA: 0s - loss: 1.5094 - Note_loss: 0.4123 - Offset_loss: 0.0316 - Volume_loss: 0.4728 - Duration_loss: 0.5926

47/47 [==============================] - 2s 40ms/step - loss: 1.5094 - Note_loss: 0.4123 - Offset_loss: 0.0316 - Volume_loss: 0.4728 - Duration_loss: 0.5926
Epoch 149/400
47/47 [==============================] - ETA: 0s - loss: 1.4901 - Note_loss: 0.3948 - Offset_loss: 0.0299 - Volume_loss: 0.4728 - Duration_loss: 0.5926

47/47 [==============================] - 2s 34ms/step - loss: 1.4901 - Note_loss: 0.3948 - Offset_loss: 0.0299 - Volume_loss: 0.4728 - Duration_loss: 0.5926
Epoch 150/400
45/47 [===========================>..] - ETA: 0s - loss: 1.5063 - Note_loss: 0.4065 - Offset_loss: 0.0343 - Volume_loss: 0.4726 - Duration_loss: 0.5929

KeyboardInterrupt: ignored

In [30]:
#prend une séquence de notes, volume, durées
pattern_note = input_note[0]
pattern_offset = input_offset[0]
pattern_volume = input_volume[0]
pattern_duration = input_duration[0]
pattern_step = input_step[0]

In [31]:
pattern_volume

array([90., 90., 65., 65., 44.,  0., 38., 51.,  0.,  0., 42.,  0.,  0.,
       42.,  0.,  0., 46.,  0., 47., 53.,  0., 44., 39., 49.,  0., 42.,
        0., 46.,  0., 44., 36., 48.,  0.,  0., 43.,  0., 44.,  0., 51.,
        0., 39., 39., 39., 35.,  0.,  0., 37., 37., 37., 33., 46., 46.,
       46., 40.,  0.,  0., 37., 37., 37., 33.,  0.,  0., 37., 37., 37.,
       33.,  0.,  0., 39., 39., 39., 46.,  0.,  0., 42., 42., 42., 47.,
       49., 49., 53.,  0.,  0., 53.,  0., 39., 39., 39., 43., 35., 35.,
       35., 49.,  0.,  0., 44., 44., 44., 45.,  0.])

In [32]:
# prediction time
prediction_output = []

for i in tqdm(range(200)):

    #on reshape les input à prédire
    note_prediction_input = numpy.reshape(pattern_note, (1, len(pattern_note), -1))
    volume_prediction_input = numpy.reshape(pattern_volume, (1, len(pattern_volume), -1))
    duration_prediction_input = numpy.reshape(pattern_duration, (1, len(pattern_duration), -1))
    offset_prediction_input = numpy.reshape(pattern_offset, (1, len(pattern_offset), 1))
    step_prediction_input = numpy.reshape(pattern_step, (1, len(pattern_step), 1))

    #prédit ici
    prediction = model.predict([note_prediction_input, offset_prediction_input, volume_prediction_input, duration_prediction_input], verbose=0)

    # prédis la note en récupérant l'index max du softmax et en faisant la transofr inverse
    # à partir du one hot train sur les notes

    temperature = 1.0

    note_softmax = prediction[0]
    adjusted_softmax = note_softmax / temperature
    adjusted_softmax = adjusted_softmax / np.sum(adjusted_softmax)
    note_soft = tf.random.categorical(note_softmax, num_samples=1)[0][0]
    notes_pred = np.zeros(len(pattern_note[0]))
    #notes_pred[np.argmax(adjusted_softmax)] = 1
    notes_pred[np.argmax(prediction[0])] = 1
    #notes_pred[note_soft] = 1
    result_note = oh_notes.inverse_transform(notes_pred.reshape(1, -1))
    pattern_note = numpy.concatenate([pattern_note, [notes_pred]])
    pattern_note = pattern_note[1:]

    """
    offset_predict = prediction[1][0][0]
    pattern_offset = numpy.concatenate([pattern_offset, prediction[1][0]])
    pattern_offset = pattern_offset[1:]
    """
    step_predict = prediction[1][0][0]
    pattern_step = numpy.concatenate([pattern_step, prediction[1][0]])
    pattern_step = pattern_step[1:]

    # la même avec le volume
    volume_pred = prediction[2][0]
    pattern_volume = numpy.concatenate([pattern_volume, volume_pred])
    pattern_volume = pattern_volume[1:]

    # la même avec la durée
    duration_pred = prediction[3][0]
    pattern_duration = numpy.concatenate([pattern_duration, duration_pred])
    pattern_duration = pattern_duration[1:]


    #on ajoute la note, le volyme et la durée
    prediction_output.append([result_note[0][0], step_predict, volume_pred[0], duration_pred[0]])


100%|██████████| 200/200 [01:17<00:00,  2.57it/s]


In [33]:
prediction_output

[['REST', 0.22386637, 1.0, 1.0],
 ['G#3', 0.66811013, 1.0, 1.0],
 ['B-3', 1.776645, 1.0, 1.0],
 ['B-3', 1.6350513, 1.0, 1.0],
 ['D5', 0.943339, 1.0, 1.0],
 ['B-3', 2.162192, 1.0, 1.0],
 ['B-2', 2.1089406, 1.0, 1.0],
 ['B-3', 1.5930723, 1.0, 1.0],
 ['B-3', 1.3491198, 1.0, 1.0],
 ['B-3', 1.5359036, 1.0, 1.0],
 ['B-4', 1.0302701, 1.0, 1.0],
 ['D5', 1.8756928, 1.0, 1.0],
 ['B-3', 1.3154297, 1.0, 1.0],
 ['A3', 1.2519398, 1.0, 1.0],
 ['B-3', 1.0387037, 1.0, 1.0],
 ['B-3', 1.1741891, 1.0, 1.0],
 ['B-3', 1.0284901, 1.0, 1.0],
 ['B-3', 0.8946203, 1.0, 1.0],
 ['B-4', 0.65488887, 1.0, 1.0],
 ['D5', 1.4257023, 1.0, 1.0],
 ['B-3', 0.84317386, 1.0, 1.0],
 ['B-3', 1.0127001, 1.0, 1.0],
 ['B-3', 0.66628873, 1.0, 1.0],
 ['B-3', 0.83090985, 1.0, 1.0],
 ['B-3', 0.86165154, 1.0, 1.0],
 ['B-3', 0.845479, 1.0, 1.0],
 ['B-4', 0.78490984, 1.0, 1.0],
 ['E-5', 1.44296, 1.0, 1.0],
 ['B-3', 1.299225, 1.0, 1.0],
 ['B-3', 1.0615107, 1.0, 1.0],
 ['B-4', 0.66621137, 1.0, 1.0],
 ['D5', 1.2412536, 1.0, 1.0],
 ['B-3', 0

In [ ]:
from music21.duration import Duration


In [ ]:
offset = 0
output_notes = []
# on passes des prédicitons à un format écoutable à l'oreille

prev_start = 0
# pour chaque note prédite
for note_p in prediction_output:
    pattern = note_p[0]
    # si note = chord
    if ('+' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('+')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = prev_start + note_p[1]
        # met le volume associé
        new_chord.volume.velocity = note_p[2]
        # met la durée associée
        new_chord.duration = Duration(note_p[3])
        output_notes.append(new_chord)
    # si note est un rest
    elif('REST'in pattern):
      note_rest = note.Rest()
      note_rest.offset = prev_start + note_p[1]
      # durée associée (pas de volume car silencieux)
      note_rest.duration = Duration(note_p[3])
      output_notes.append(note_rest)
    else:
        new_note = note.Note(pattern)
        new_note.offset = prev_start + note_p[1]
        new_note.volume.velocity = note_p[2]
        new_note.duration = Duration(note_p[3])
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    prev_start = prev_start + note_p[1]
    # increase offset each iteration so that notes do not stack
    #offset += 0.5

In [ ]:
#into midi
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_outputminecraft6.mid')

'test_outputminecraft6.mid'